<a href="https://colab.research.google.com/github/GuiifCorso/Alura-Gemini/blob/main/AI'Eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playsound --upgrade
!pip install pillow
!pip install gtts
!pip install google.generativeai
!pip install googletrans==4.0.0-rc1

In [32]:
from googletrans import Translator
import os

#Coletando a língua-mãe do usuário para tornar o "aplicativo" mais acessível
language = os.environ.get("LANG")

lang_ab = language.split('.')[0]
#Como a biblioteca "os" retorna a língua como en_US.UTF-8, precisa-se separar somente a língua, que é o que estou fazendo abaixo:
lang = lang_ab[:2]
#Mensagem base de início
base_message = "Qual site você estará acessando? "


#Definição do tradutor de texto para a língua no computador do usuário
def translate_text(text, target_language='pt'):
  translator = Translator()
  translated_text = translator.translate(text, dest=target_language)
  return translated_text.text

#Nova mensagem base traduzida
new_base_message = translate_text(base_message, lang)
print(new_base_message)

À quel site allez-vous accéder?


In [33]:
from typing import SupportsRound
from gtts import gTTS
from IPython.display import Audio
from IPython.core.display import display

#Aqui estarei criando um áudio, na língua-mãe do usuário, com o comando para o primeiro passo da utilização do software: Perguntar o site
def audio_maker(text, lang, name):
  tts = gTTS(text=text, lang=lang)
  tts.save(f"{name}.wav")


audio_maker(new_base_message, lang, f"acess-{lang}")

def audio_play(audio):
  display(Audio(audio, autoplay=True))


#OBS: Estou aguardando a biblioteca playsound resolver algum problema que ela está tendo, pois não consigo reproduzir áudios. Se o problema continuar não estarei colocando a função de áudio :(

In [ ]:
from ast import Suite
from IPython.display import Audio
import google.generativeai as genai
from google.colab import userdata
import time

#Perguntando qual site o usuário gostaria de acessar
audio_play(f"acess-{lang}.wav")
time.sleep(5)
print(new_base_message)
site = input()

api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

prompt_parts = [
  "input: globo",
  "output: https://g1.globo.com",
  "input: youtube",
  "output: https://youtube.com",
  "input: github",
  "output: https://github.com",
  "input: yt",
  "output: https://youtube.com",
  "input: whatsapp",
  "output: https://whatsapp.com",
  "input: alura",
  "output: https://alura.com",
  "input: udemy",
  "output: https://udemy.com",
  "input: times",
  "output: https://nytimes.com",
  f"input: {site}",
  "output: ",
]

response = model.generate_content(prompt_parts)
print(response.text)

#A IA que construí acima, recebe o nome do site que o usuário quer acessar e transforma o valor em um link para que possa ser buscado no código seguinte
url = response.text
url = url.strip()

In [35]:
import requests as r
from bs4 import BeautifulSoup as bs
import os


#Utilizando a biblioteca requests e beautiful soup para entrar na página que o usuário quer
response = r.get(url)
soup = bs(response.content, "html.parser")

image_list = []

i = 0
for image in image_list:
  i += 1
  os.remove(f"/content/image{i}.jpg")

#Utilizando o beautifulsoup para encontrar todos os elementos "img" da página
images = soup.find_all("img", attrs={"src": True})

index = 0

#Para cada imagem encontrada, repetir o código abaixo:
for image in images:
    index += 1
    #Aqui coleto o URL "cru" da imagem, que não consegue ser lido pela IA
    image_raw_url = image["src"]
    #Aqui meio que "abro" o URL e acesso seus dados
    if "https" in image_raw_url:
      image_url = r.get(image_raw_url, stream=True)
    else:
      pass
    #Atribuo à variável "image" o .content do URL, o que me dá os dados da imagem
    image = image_url.content
    #Com os dados da imagem, agora posso abrí-la em um arquivo, que chamarei de imagem[x], onde x é o número da imagem
    with open(f"image{index}.jpg", "wb") as f:
        f.write(image)
    image_list.append(image_url)

index = 0

In [36]:
from pathlib import Path
import hashlib
import google.generativeai as genai


genai.configure(api_key=api_key)

#Iniciando o modelo responsável pela descrição das imagens
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Seja direto e curto, não enrole demais. Foque nos detalhes e na descrição do que estiver sendo pedido.\nVocê atuará como um descritor de imagens para deficientes visuais, então deve analisar a imagem e descrevê-la em pouco menos de 300 palavras com o maior grau de entendimento."

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

convo = model.start_chat(history=[
])


In [ ]:
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata
import time

#Mensagem de continuação
follow_message = "Deseja continuar? Y para sim, N para não"
translated_follow_message = translate_text(follow_message, lang)
audio_maker(translated_follow_message, lang, f"follow-{lang}")


#Mensagem de erro
error_message = "Erro. Digite uma tecla válida"
translated_error_message = translate_text(error_message, lang)
audio_maker(translated_error_message, lang, f"error-{lang}")

#Mensagem de despedida
bye_message = "Muito obrigado! Até a próxima..."
translated_bye_message = translate_text(bye_message, lang)
audio_maker(translated_bye_message, lang, f"bye-{lang}")

#Para cada imagem encontrada, novamente, mas agora em uma lista, para deixá-las armazenadas
for image in image_list:
  loop = True
  index += 1

  path = f"/content/image{index}.jpg"

  prompt = [genai.upload_file(path), f"Descreva esta imagem na língua {language}:"]
  #A IA irá descrever cada imagem de acordo com o loop e a posição das imagens com base nas imagens .jpg criadas
  convo.send_message(prompt)
  print(convo.last.text)

  #Criação e play dos áudios da descrição
  audio_maker(convo.last.text, lang, f"desc-{index}")
  audio_play(f"desc-{index}.wav")

  print("-"*50)

  time.sleep(60)

  #Opção de seguir ou continuar a descrever
  audio_play(f"follow-{lang}.wav")
  time.sleep(5)
  print(translated_follow_message)


  while loop:
    follow = input().lower()
    if follow == "y":
      #Caso queira continuar
      print("-"*50)
      loop = False
      continue
    elif follow == "n":
      break
    else:
      #Caso dê erro:
      audio_play(f"error-{lang}.wav")
      print(translated_error_message)
      continue
  if follow == "n":
    #Caso não queira mais continuar
    audio_play(f"bye-{lang}.wav")
    print(translated_bye_message)
    break
